### 🥊 Proof of Concept: RNN + GNN for UFC Fight Predictions

This prototype demonstrates how an RNN can encode round-by-round fighter performance, and a GNN can use those embeddings to predict fight outcomes.

- **Fighters:** 4 synthetic fighters with differing abilities (strong, average, weak, etc.).  
- **Round Data:** Each fighter has per-round stats (strikes landed, strikes absorbed), with added noise to emulate realistic, non-deterministic fight outcomes.  
- **Architecture:**  
  - **RNN Encoder:** Processes sequential round-by-round stats for each fighter.  
  - **GNN:** Takes the fighter node features and edge embeddings (from RNN) to predict win probabilities.  
- **Training:** Epochs have been kept low for this proof of concept, demonstrating that the model can start to learn patterns without overfitting on tiny synthetic data.  

This setup is intended as a starting point for experimenting with more fighters, real UFC data, and richer fight features.


In [ ]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv

# -----------------------------
# 1. Synthetic fighter nodes
# -----------------------------
def noisy_rounds(base_stats, rounds=3, noise_level=3.0):
    # base_stats: [strikes_landed, strikes_absorbed]
    rounds_list = []
    for _ in range(rounds):
        strikes_landed = max(0, base_stats[0] + random.gauss(0, noise_level))
        strikes_absorbed = max(0, base_stats[1] + random.gauss(0, noise_level))
        rounds_list.append([strikes_landed, strikes_absorbed])
    return torch.tensor(rounds_list, dtype=torch.float)

# Fighter nodes: 0, 1, 2, 3
fighter_features = torch.tensor([
    [180, 70, 28],  # Fighter 0     good
    [175, 68, 30],  # Fighter 1     ok
    [185, 72, 26],  # Fighter 2     great
    [178, 69, 27],  # Fighter 3     bad
], dtype=torch.float)

# Synthetic fights: rounds x 2 features per fighter
fight_data = {
    (0,1): {
        0: noisy_rounds([10,6]),  # Fighter 0
        1: noisy_rounds([9,7]),   # Fighter 1
        'winner': 0
    },
    (0,2): {
        0: noisy_rounds([10,6]),
        2: noisy_rounds([12,5]),
        'winner': 2
    },
    (0,3): {
        0: noisy_rounds([10,6]),
        3: noisy_rounds([8,9]),
        'winner': 0
    },
    (1,2): {
        1: noisy_rounds([9,7]),
        2: noisy_rounds([12,5]),
        'winner': 2
    },
    (1,3): {
        1: noisy_rounds([9,7]),
        3: noisy_rounds([8,9]),
        'winner': 1
    },
    (2,3): {
        2: noisy_rounds([12,5]),
        3: noisy_rounds([8,9]),
        'winner': 2
    },
}


# -----------------------------
# 3. RNN encoder for round sequences
# -----------------------------
class FighterRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, h_n = self.rnn(x)  # [1, batch, hidden_dim]
        out = self.fc(h_n.squeeze(0))
        return out

rnn_input_dim = 2
rnn_hidden_dim = 8
edge_embedding_dim = 4
rnn_encoder = FighterRNN(rnn_input_dim, rnn_hidden_dim, edge_embedding_dim)

# -----------------------------
# 4. GNN model
# -----------------------------
class FightGNN(nn.Module):
    def __init__(self, node_input_dim, node_hidden_dim, edge_input_dim):
        super().__init__()
        self.conv1 = GCNConv(node_input_dim, node_hidden_dim)
        self.conv2 = GCNConv(node_hidden_dim, node_hidden_dim)
        self.edge_mlp = nn.Sequential(
            nn.Linear(node_hidden_dim*2 + edge_input_dim, node_hidden_dim),
            nn.ReLU(),
            nn.Linear(node_hidden_dim, 1)
        )

    def forward(self, x, edge_index, edge_attr):
        # Node embeddings
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))

        # Edge-level prediction using node embeddings + edge_attr
        src_nodes = edge_index[0]
        dst_nodes = edge_index[1]
        edge_input = torch.cat([x[src_nodes], x[dst_nodes], edge_attr], dim=-1)
        edge_preds = self.edge_mlp(edge_input)
        return torch.sigmoid(edge_preds).squeeze()

gnn = FightGNN(node_input_dim=3, node_hidden_dim=16, edge_input_dim=edge_embedding_dim*2)

# -----------------------------
# 5. Prepare edges and labels
# -----------------------------
def encode_edges(fight_data, rnn_encoder):
    edge_index = []
    edge_attr_list = []
    edge_labels = []
    for (f1,f2), fight in fight_data.items():
        for src, dst in [(f1,f2),(f2,f1)]:  # bidirectional edges
            edge_index.append([src,dst])
            seq_src = fight[src].unsqueeze(0)
            seq_dst = fight[dst].unsqueeze(0)
            emb_src = rnn_encoder(seq_src)
            emb_dst = rnn_encoder(seq_dst)
            edge_attr = torch.cat([emb_src, emb_dst], dim=-1)
            edge_attr_list.append(edge_attr.squeeze(0))
            edge_labels.append(torch.tensor(fight['winner']==src, dtype=torch.float))
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous(), torch.stack(edge_attr_list), torch.stack(edge_labels)

edge_index, edge_attr, edge_labels = encode_edges(fight_data, rnn_encoder)

# -----------------------------
# 6. Training loop
# -----------------------------
optimizer = optim.Adam(list(gnn.parameters()) + list(rnn_encoder.parameters()), lr=0.01)
criterion = nn.BCELoss()
num_epochs = 50

gnn.train()
rnn_encoder.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    edge_index_train, edge_attr_train, edge_labels_train = encode_edges(fight_data, rnn_encoder)
    preds = gnn(fighter_features, edge_index_train, edge_attr_train)
    loss = criterion(preds, edge_labels_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# -----------------------------
# 7. Evaluation
# -----------------------------

gnn.eval()
rnn_encoder.eval()

with torch.no_grad():
    print("\n--- Fight Predictions ---\n")
    
    for (f1, f2), fight in fight_data.items():
        # Encode each fighter's rounds
        seq_f1 = fight[f1].unsqueeze(0)
        seq_f2 = fight[f2].unsqueeze(0)
        emb_f1 = rnn_encoder(seq_f1)
        emb_f2 = rnn_encoder(seq_f2)
        
        # Combine embeddings for edge features
        edge_attr = torch.cat([emb_f1, emb_f2], dim=-1)
        
        # Predict probability that fighter 1 (f1) wins
        pred = gnn(fighter_features, torch.tensor([[f1, f2],[f2, f1]], dtype=torch.long), 
                   torch.stack([edge_attr.squeeze(0), edge_attr.squeeze(0)]))
        
        # Take first edge prediction as representative
        win_prob_f1 = pred[0].item()
        winner_idx = fight['winner']
        
        print(f"Fight: Fighter {f1} vs Fighter {f2}")
        print(f"  Predicted win prob (Fighter {f1} wins): {win_prob_f1:.4f}")
        print(f"  True winner: Fighter {winner_idx}\n")


Epoch 10, Loss: 0.6955
Epoch 20, Loss: 0.6896
Epoch 30, Loss: 0.6467
Epoch 40, Loss: 0.4347
Epoch 50, Loss: 0.1259

--- Fight Predictions ---

Fight: Fighter 0 vs Fighter 1
  Predicted win prob (Fighter 0 wins): 0.9377
  True winner: Fighter 0

Fight: Fighter 0 vs Fighter 2
  Predicted win prob (Fighter 0 wins): 0.3678
  True winner: Fighter 2

Fight: Fighter 0 vs Fighter 3
  Predicted win prob (Fighter 0 wins): 0.9862
  True winner: Fighter 0

Fight: Fighter 1 vs Fighter 2
  Predicted win prob (Fighter 1 wins): 0.0416
  True winner: Fighter 2

Fight: Fighter 1 vs Fighter 3
  Predicted win prob (Fighter 1 wins): 0.9526
  True winner: Fighter 1

Fight: Fighter 2 vs Fighter 3
  Predicted win prob (Fighter 2 wins): 0.9864
  True winner: Fighter 2

